In this notebook we introduce reinforcement learning formalised as a Markov Decision Process.

# Markov Decision Processes

**State**: $s_t \in \mathbb{S}$

**Action**: $a_t \in \mathbb{A}$

**Reward**: $r_t \in \mathbb{R}$

**Policy**: $\Pi_k(a|s) = p(a_k = a|s_k = s)$

**Model**: $p(s', r|s, a)$

**Objective**: $\max_{\mathbf{a}}\sum_{k=0}^\infty \lambda^k r(s,a)$

**Markove Property**: $p(s_{k+1} = s', r_k = r|s_k, a_k, r_{k-1}, s_{k-1}, a_{k-1},...) = p(s_{k+1} = s', r_k = r|s_k, a_k)$


## Value Iteration

the **objective** can be rewritten as 

$$\min_{\mathbf{u}}\sum_{k=0}^\infty \lambda^k c_k(x_k, u_k)$$

Referring back to [`1_optimal_control.ipynb`](1_optimal_control.ipynb) we can easily show the new Gellman equation is

$$V(x, k) = \min_u(c(x, u) + \lambda V(f(x, u), k + 1)), \quad k = h-1, h-2, ..., 1, 0$$

where $\lambda \leq 1$ is the discount factor, lower indicates we prioritise ealier rewards

Episodic problems are finite horizon which stop when $\forall x \in X_s$ (stopping set X_s):

- $f(x,u) \in X_s$
- c(x, u) = 0

As a defined stopping set is given, we don't require separate values for the same state at different times, hence yielding

$$V(x) = \min_u(c(x, u) + \lambda V(f(x, u)))$$

solved by

$$V_{k+1}(x) = \min_u(c(x, u) + \lambda V_k(f(x, u)))$$

**Proof of Convergence** $(\lambda < 1)$:

$$
\begin{align*}
|BV_1(x) - BV_2(x)| &= |\min_u(c(x, u) + \lambda V_1(f(x, u))) - \min_u(c(x, u) + \lambda V_2(f(x, u)))| \\
&\leq \max_u|\lambda V_1(f(x,u))- \lambda V_2(f(x,u))| \\
&= \lambda \max_u| V_1(f(x,u))-  V_2(f(x,u))|

\end{align*}
$$


## Policy Iteration

For policy $u=\Pi(x)$ the value function is

$$V^\Pi(x) = \min_u(c(x, \Pi(x)) + \lambda V^\Pi(f(x, \Pi(x))))$$

1. Initalise Policy $\Pi$
2. Compute $V^\Pi$ til convergence
3. Update $\Pi$ to greedy w.r.t. $V^\Pi$

    $$\Pi(x) = \argmin_u(c(x, \Pi(x)) + \lambda V^\Pi(f(x, \Pi(x))))$$

4. Repeat 2 and 3 til policy converges

**Theorem**:

$$
\begin{align*}
V^\Pi(x) &\geq \min_u \left( c(x, \Pi'(x)) + \lambda V^\Pi(f(x, \Pi '(x))) \right) \\
&= V^{\Pi '}(x)
\end{align*}
$$


## MDP Case

The theory above applied to a MDP with a stochastic policy yields

$$V^\Pi(s) = \sum_a \Pi(a|s) \sum_{s' \in S} \sum_{r \in R} p(s', r|s, a)(r + \lambda V^\Pi(s'))$$

The optimal is hence

$$V(s) = \max_a \sum_{s' \in S} \sum_{r \in R} p(s', r|s, a)(r + \lambda V(s'))$$

In practice, learning the value function from samples in not usually done. Instead we use the action-value function defined as

$$Q(s, a) = \sum_{s' \in S} \sum_{r \in R} p(s', r|s, a)(r + \lambda V(s'))$$

The link to the optimal value function above is clearly seen, with the optimal action similarly determined.

The recursion property is also maintained

$$Q(s, a) = \sum_{s' \in S} \sum_{r \in R} p(s', r|s, a)(r + \lambda \max_b Q(s', b))$$

## Monte Carlo Exploring Starts

A simple MC algorithm is possible with the value function, but is slow to converge as many samples are required for each policy, of which there may be many. **MCES** instead uses the Q function.

Repeat:

- Choose random start $s_0$ and action $a_0$, then simulate trajectory  until $s_n \in X_s$:
    - $r_0, s_1 \sim  p(s_1, r_0|s_0, a_0)$
    - $a_1 = \argmax_a Q(s_1, a)$


- Let $R_n = r_n, \quad R_k = r_k + λR_{k+1}, \quad k = n-1,...,0$

- Update Q values for k = 0,...,n:
$$\begin{align*} Q(s_k,a_k) \leftarrow& \frac{N(s_k,a_k)-1}{N(s_k,a_k)} Q(s_k,a_k) + \frac{1}{N(s_k,a_k)} R_k \\
 \leftarrow& Q(s_k,a_k) + \frac{1}{N(s_k,a_k)}(R_k - Q(s_k,a_k))\end{align*}$$


N(s,a) is number of times state-action pair s,a. 

There are three approaches to deciding when to update the Q for a state-action pair:
1. Update each time it is visited (Gives biased estimates because of revisits)
2. Just update $s_0, a_0$ (Wastes lots of samples)
3. Update each time a state-action pair is visited for the first time.

Unlike the base MC method, **MCES** has not yet been proven to converge.

## Temporal Difference Mehtods

TD methods update after each sample unlike MC methods which update after a full trajectory has been sampled. This means that after each sample, the update ensures the next samples are more optimal as the information revealed by the previous step has already been implemented

### Q-learning

This is an off-policy method as the values of the optimal policy can be obtained via any policy such as

$$\pi(s) = \begin{cases}
\arg\max_a Q(s,a), & \text{with probability } 1-\epsilon \\
\text{randomly selected}, & \text{with probability } \epsilon
\end{cases}$$

After each sample we update the Q function using 

$$Q_{k+1}(s,a) = (1-\alpha_k)Q_k(s,a) + \alpha_k(r + \lambda \max_b Q_k(s',b))$$



convergence is given by

$$ \{N(s,a) = \infty, \forall s \in \mathbb{S}, \forall a\in \mathbb{A}\},\quad \sum \alpha_k = \infty, \quad \sum \alpha^2_k < 0 \implies  Q_k \rightarrow Q_{optimal}$$

If deterministic $\alpha_k = 1$ will converge to0.

An example is given in [`QLearning.py`](QLearning.py).

### SARSA

SARSA is an on-policy method as it assumes finite actions are obtained with the current policy.

$$Q_{k+1}^\pi(s,a) = (1-\alpha_k)Q_k^\pi(s,a) + \alpha_k(r + \lambda Q_k^\pi(s',\pi(s')))$$

Converges when $ \{N(s,a) = \infty, \forall s \in \mathbb{S}, \forall a\in \mathbb{A}\}$ and the policies converge to the greeedy policy.

The update method given in [`QLearning.py`](QLearning.py) can be altered to use the policy instead of 'np.max(self.q_table[next_state])' to yield the SARSA method.